# Count quotes and words for the first table

## Count total number of quotes

These come from the "CBC_qt_output/Quotes_new" dir. This is the 'last' place where we can count quotes, as in the sentiment output, the quotes for each article are merged into one document.

In [1]:
import pandas as pd
import numpy as np
import json
from pandas import json_normalize
import ast
from ast import literal_eval
import os
import glob
import re
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Counting number of quotes
We count number of quotes from files in 'CBC_qt_output/Quotes_new' dir, as this is the last place where we can count quotes individually (in future files, they are merged by article). **Important:** Because Quotes_new only contains files that were re-processed by using an online csv -> xlsx converter, and Feb and Jun were already previously processed in this manner, this folder does not contain new files for Feb and Jun. Instead, these files, named '2023_02_feb_CBC_news_qt.xlsx' and '2023_06_jun_CBC_news_qt.xlsx' live in the folder 'CBC_qt_output/old'.

In [5]:
directory = '/content/drive/My Drive/evaluation_quotes/CBC_input'
excel_files = glob.glob(os.path.join(directory, '2023*.xlsx'))
quotes = []

for file in excel_files:
    try:
        # read the full sheet and the quotes column
        df = pd.read_excel(file, sheet_name='full')

        # count the number of rows
        num_rows_quotes = len(df)

        # Store the result in the list
        quotes.append({
            'file': os.path.basename(file),
            'rows_in_full': num_rows_quotes,
        })

    except Exception as e:
        # Handle any errors (e.g., sheet not found)
        print(f"Could not process {file}: {e}")

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:232: UserWarning: Workbook contains no stylesheet, using openpyxl's defaults
  warn("Workbook contains no stylesheet, using openpyxl's defaults")
/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:232: UserWarning: Workbook contains no stylesheet, using openpyxl's defaults
  warn("Workbook contains no stylesheet, using openpyxl's defaults")
/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:232: UserWarning: Workbook contains no stylesheet, using openpyxl's defaults
  warn("Workbook contains no stylesheet, using openpyxl's defaults")
/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:232: UserWarning: Workbook contains no stylesheet, using openpyxl's defaults
  warn("Workbook contains no stylesheet, using openpyxl's defaults")
/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:232: UserWarning: Workbook contains no stylesheet, using openpyxl'

In [6]:
quotes_df = pd.DataFrame(quotes)

quotes_df

,file,rows_in_full
0,2023_01_jan_CBC_news_qt_new.xlsx,55161
1,2023_02_feb_CBC_news_qt.xlsx,53574
2,2023_06_jun_CBC_news_qt.xlsx,55454
3,2023_04_apr_CBC_news_qt_new.xlsx,51383
4,2023_05_may_CBC_news_qt_new.xlsx,58306
5,2023_07_jul_CBC_news_qt_new.xlsx,51754
6,2023_08_aug_CBC_news_qt_new.xlsx,51011
7,2023_09_sep_CBC_news_qt_new.xlsx,52230
8,2023_10_oct_CBC_news_qt_new.xlsx,55202
9,2023_11_nov_CBC_news_qt_new.xlsx,55528


In [8]:
# Calculate the total row by summing all numeric columns
total_row = quotes_df[['rows_in_full']].sum()

# Add 'Total' as a label for the 'Category' column in the total row
total_row['file'] = 'Total'

# Append the total row to the DataFrame
quotes_df = pd.concat([quotes_df, pd.DataFrame([total_row])], ignore_index=True)

# Display the updated DataFrame
quotes_df

,file,rows_in_full
0,2023_01_jan_CBC_news_qt_new.xlsx,55161
1,2023_02_feb_CBC_news_qt.xlsx,53574
2,2023_06_jun_CBC_news_qt.xlsx,55454
3,2023_04_apr_CBC_news_qt_new.xlsx,51383
4,2023_05_may_CBC_news_qt_new.xlsx,58306
5,2023_07_jul_CBC_news_qt_new.xlsx,51754
6,2023_08_aug_CBC_news_qt_new.xlsx,51011
7,2023_09_sep_CBC_news_qt_new.xlsx,52230
8,2023_10_oct_CBC_news_qt_new.xlsx,55202
9,2023_11_nov_CBC_news_qt_new.xlsx,55528


## Counting number of words in quotes and non-quotes
Input dir is sentiment_output. We count the words in the following tabs/columns of each file; i.e. for each month:
* quotes tab - quote column
* non_quotes tab - non_quoted_text column

In doing this by accessing files in my Google Drive, I split the text files in the output corpus into 'quotes' and 'non_quotes' folders to make accessing them easier.

Paths are 'CBC_output_corpus/quotes' and 'CBC_output_corpus/non_quotes' respectively

In [9]:
quotes_dir = '/content/drive/My Drive/evaluation_quotes/CBC_output_corpus/quotes'
non_quotes_dir = '/content/drive/My Drive/evaluation_quotes/CBC_output_corpus/non_quotes'

In [13]:
quote_txts = glob.glob(os.path.join(quotes_dir, '*.txt'))

# Initialize a variable to store the total word count
quotes_wc = 0

for fp in quote_txts:
  with open(fp, 'r', encoding='utf-8') as qt_txt:
    lines = qt_txt.readlines()

    # convert the lines to a df
    df = pd.DataFrame(lines, columns=['text'])

    # count the number of words in each line and sum them
    df['word_count'] = df['text'].apply(lambda x: len(x.split()))
    quotes_wc += df['word_count'].sum()

# print total word count
print(f"Total number of words in all 'quotes' related text files: {quotes_wc}")

Total number of words in all 'quotes' related text files: 10873667


In [11]:
non_quote_txts = glob.glob(os.path.join(non_quotes_dir, '*.txt'))

# Initialize a variable to store the total word count
non_quotes_wc = 0

for fp in non_quote_txts:
  with open(fp, 'r', encoding='utf-8') as qt_txt:
    lines = qt_txt.readlines()

    # convert the lines to a df
    df = pd.DataFrame(lines, columns=['text'])

    # count the number of words in each line and sum them
    df['word_count'] = df['text'].apply(lambda x: len(x.split()))
    non_quotes_wc += df['word_count'].sum()

# print total word count
print(f"Total number of words in all 'non_quotes' related text files: {non_quotes_wc}")

Total number of words in all 'non_quotes' related text files: 17314398


In [15]:
print("Total number of words in quotes and non-quotes: ", str(quotes_wc + non_quotes_wc))

Total number of words in quotes and non-quotes:  28188065


## Counting original corpus for words
Because total word count also includes speakers and verbs

In [16]:
input = '/content/drive/My Drive/evaluation_quotes/CBC_input_correct_encoding'
xlsx_files = glob.glob(os.path.join(input, '*.xlsx'))

total_word_count = 0

for fp in xlsx_files:
  # read excel file
  df = pd.read_excel(fp, usecols=['body'])

  # count words and add to total
  df['body'] = df['body'].apply(lambda x: len(str(x).split()) if pd.notnull(x) else 0)
  total_word_count += df['body'].sum()

# print total word count
print(f"Total number of words in body across all xlsx files: {total_word_count}")

# compare with number of words in quotes and non quotes
print("Total number of words in quotes and non-quotes: ", str(quotes_wc + non_quotes_wc))

Total number of words in body across all xlsx files: 32287506
Total number of words in quotes and non-quotes:  28188065


# Initial attempt to count quotes and words


## Count number of quotes

In [ ]:
directory = 'C:/Maite/MOD/projects/Monika_Bednarek/Evaluation_quotes/Data/CBC_qt_output/Quotes_new'

In [ ]:
excel_files = glob.glob(os.path.join(directory, '2023*.xlsx'))

results = []

for file in excel_files:
    try:
        # read the full sheet and the quotes column
        df = pd.read_excel(file, sheet_name='full')

        # count the number of rows
        num_rows_quotes = len(df)

        # Store the result in the list
        results.append({
            'file': os.path.basename(file),
            'rows_in_full': num_rows_quotes,
        })

    except Exception as e:
        # Handle any errors (e.g., sheet not found)
        print(f"Could not process {file}: {e}")

In [ ]:
results_df = pd.DataFrame(results)

results_df

,file,rows_in_full
0,2023_01_jan_CBC_news_qt_new.xlsx,55161
1,2023_03_mar_CBC_news_qt_new.xlsx,60157
2,2023_04_apr_CBC_news_qt_new.xlsx,51383
3,2023_05_may_CBC_news_qt_new.xlsx,58306
4,2023_07_jul_CBC_news_qt_new.xlsx,51754
5,2023_08_aug_CBC_news_qt_new.xlsx,51011
6,2023_09_sep_CBC_news_qt_new.xlsx,52230
7,2023_10_oct_CBC_news_qt_new.xlsx,55202
8,2023_11_nov_CBC_news_qt_new.xlsx,55528
9,2023_12_dec_CBC_news_qt_new.xlsx,47561


In [ ]:
# Calculate the total row by summing all numeric columns
total_row = results_df[['rows_in_full']].sum()

# Add 'Total' as a label for the 'Category' column in the total row
total_row['file'] = 'Total'

# Append the total row to the DataFrame
results_df = results_df.append(total_row, ignore_index=True)

# Display the updated DataFrame
results_df

,file,rows_in_full
0,2023_01_jan_CBC_news_qt_new.xlsx,55161
1,2023_03_mar_CBC_news_qt_new.xlsx,60157
2,2023_04_apr_CBC_news_qt_new.xlsx,51383
3,2023_05_may_CBC_news_qt_new.xlsx,58306
4,2023_07_jul_CBC_news_qt_new.xlsx,51754
5,2023_08_aug_CBC_news_qt_new.xlsx,51011
6,2023_09_sep_CBC_news_qt_new.xlsx,52230
7,2023_10_oct_CBC_news_qt_new.xlsx,55202
8,2023_11_nov_CBC_news_qt_new.xlsx,55528
9,2023_12_dec_CBC_news_qt_new.xlsx,47561


## Count words in quotes and non quotes

Input dir is sentiment_output. We count the words in the respective tab/columns of each excel, i.e, for each month:

* quotes tab
* quote column
* non-quotes tab
* non-quoted text

In [ ]:
os.chdir(r'C:\Maite\MOD\projects\Monika_Bednarek\Evaluation_quotes\Data\CBC_output_corpus')

This one counts with the \n

In [ ]:
with open('2023_02_feb_CBC_news_sentiment_quote_text.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

df = pd.DataFrame(lines, columns=['text'])

# count the number of words in each line
df['word_count'] = df['text'].apply(lambda x: len(x.split()))

total_words = df['word_count'].sum()

print(f"Total number of words: {total_words}")

Total number of words: 895688


This one without the \n

In [ ]:
with open('2023_02_feb_CBC_news_sentiment_quote_text.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()
    # Remove the newline character and strip any extra whitespace from each line
    lines = [line.strip().replace('\n', '') for line in lines]

df = pd.DataFrame(lines, columns=['text'])

# count the number of words in each line
df['word_count'] = df['text'].apply(lambda x: len(x.split()))

total_words = df['word_count'].sum()

print(f"Total number of words: {total_words}")

Total number of words: 895688


From this, I conclude that somehow pandas doesn't see the \n

In [ ]:
print(df)

                                                   text  word_count
0     Wednesday's order from the 6th U.S. Circuit Co...         182
1     that thousands of people were travelling acros...         146
2     13 state landmarks — including One World Trade...          94
3     it's going to take a lot more than talk to put...         617
4     We're in a situation where we have members say...          26
...                                                 ...         ...
4211  that while more government money is needed, th...         807
4212  "A dollar invested today, and making sure you ...         314
4213  he had accepted the resignation of Bryce Phill...          78
4214  she felt things were going well right up until...         543
4215  the heaviest snow will be centred in the north...          43

[4216 rows x 2 columns]


In [ ]:
df.iloc[2,0]

'13 state landmarks — including One World Trade Center, Empire State Plaza and Niagara Falls — would be illuminated Monday evening in red, black and green"Amidst the recent rise in bigotry, division and hate across this country, it is more important than ever that we remember Dr. Martin Luther King Jr.\'s teachings on justice, peace, equality and loveMarching bands, dance teams, local politicians and activists made their way along a route that "traces the steps" of King during visits to Miami,"the sacred right to vote""one of the country\'s oldest and largest""I am the dream,"'

So now I'll proceed to do this for the quotes vs. non-quotes in the dir

In [ ]:
directory = 'C:/Maite/MOD/projects/Monika_Bednarek/Evaluation_quotes/Data/CBC_output_corpus'

In [ ]:
pwd

'C:\\Maite\\MOD\\projects\\Monika_Bednarek\\Evaluation_quotes\\Data\\CBC_output_corpus'

In [ ]:
# Step 2: Get all text files in the directory that contain "quote" between underscore in their name
quotes_files = [f for f in os.listdir(directory) if '_quote_' in f.lower() and f.endswith('.txt')]

# Initialize a variable to store the total word count
total_word_count = 0

# Step 3: Process each file
for file_name in quotes_files:
    file_path = os.path.join(directory, file_name)

    # Read the text file into a list of lines
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Convert the lines into a DataFrame
    df = pd.DataFrame(lines, columns=['text'])

    # Count the number of words in each line and sum them
    df['word_count'] = df['text'].apply(lambda x: len(x.split()))
    total_word_count += df['word_count'].sum()

# Step 4: Print the total word count
print(f"Total number of words in all 'quotes' related text files: {total_word_count}")


Total number of words in all 'quotes' related text files: 28188065


This is actually the total number of everyting, as I cannot tell _quote_ from _non_quote

In [ ]:
# Step 2: Get all text files in the directory that contain "quote" between underscore in their name
quotes_files = [f for f in os.listdir(directory) if '_non_quote_' in f.lower() and f.endswith('.txt')]

# Initialize a variable to store the total word count
total_word_count = 0

# Step 3: Process each file
for file_name in quotes_files:
    file_path = os.path.join(directory, file_name)

    # Read the text file into a list of lines
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Convert the lines into a DataFrame
    df = pd.DataFrame(lines, columns=['text'])

    # Count the number of words in each line and sum them
    df['word_count'] = df['text'].apply(lambda x: len(x.split()))
    total_word_count += df['word_count'].sum()

# Step 4: Print the total word count
print(f"Total number of words in all 'non quotes' related text files: {total_word_count}")


Total number of words in all 'non quotes' related text files: 17314398


## Now, I'm counting the original corpus words, because the quotes vs. non-quotes excludes verbs and speakers

In [ ]:
directory = 'C:/Maite/MOD/projects/Monika_Bednarek/Evaluation_quotes/Data/CBC_input_correct_encoding'

In [ ]:
xlsx_files = [f for f in os.listdir(directory) if f.endswith('.xlsx')]

column_name = 'body'

total_word_count = 0

for file_name in xlsx_files:
    file_path = os.path.join(directory, file_name)

    # Read the Excel file into a pandas ExcelFile object to handle multiple sheets
    xlsx_file = pd.ExcelFile(file_path)

    # Step 4: Process each sheet in the Excel file
    for sheet_name in xlsx_file.sheet_names:
        df = xlsx_file.parse(sheet_name)

        # Step 5: Check if the specified column exists in the sheet
        if column_name in df.columns:
            # Step 6: Count the number of words in the specified column (excluding NaN values)
            df[column_name] = df[column_name].apply(lambda x: len(str(x).split()) if pd.notnull(x) else 0)

            # Add up the word counts in this column
            total_word_count += df[column_name].sum()

# Step 7: Print the total word count
print(f"Total number of words in column '{column_name}' across all xlsx files: {total_word_count}")


Total number of words in column 'body' across all xlsx files: 26772464
